In [6]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame
InteractiveShell.ast_node_interactivity = "all"

In [35]:
from src.config import settings
from src.data import utils as du

import logging
import duckdb

import mlflow
from mlflow.entities import ViewType

from splink.duckdb.linker import DuckDBLinker

In [68]:
x = f"""
    models:/
    companies_matching_model.json
"""

"".join(line.strip() for line in x.splitlines())

'models:/companies_matching_model.json'

In [61]:
query = 'attribute.status = FINISHED'

x = mlflow.search_runs(
    experiment_ids=mlflow.get_experiment_by_name(
        'Default'
    ).experiment_id
)

In [55]:
x.status == 'FINISHED'

0      True
1     False
2      True
3      True
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15     True
16     True
17     True
18     True
19     True
20     True
21     True
Name: status, dtype: bool

In [60]:
x[(x.end_time == max(x.end_time)) & (x.status == 'FINISHED')].run_id[0]

'a325a3cb39d443e2a734e128ee7105b7'

In [46]:
x.columns

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.metric_1', 'metrics.metric_2', 'metrics.score',
       'metrics.r2', 'metrics.rmse', 'metrics.mae', 'params.param_1',
       'params.random_state', 'params.n_estimators', 'params.my',
       'params.alpha', 'params.l1_ratio', 'tags.git_hash',
       'tags.mlflow.runName', 'tags.mlflow.source.name', 'tags.mlflow.user',
       'tags.mlflow.source.type', 'tags.dev', 'tags.mlflow.source.git.commit',
       'tags.mlflow.note.content', 'tags.sample_tag',
       'tags.mlflow.log-model.history'],
      dtype='object')

In [32]:
[exp for exp in mlflow.search_experiments() where experiment_id]

mlflow.get_experiment_by_name

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1668086999660, experiment_id='1', last_update_time=1668086999660, lifecycle_stage='active', name='fex_v3', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1667812210812, experiment_id='0', last_update_time=1667812210812, lifecycle_stage='active', name='Default', tags={'mlflow.note.content': 'A place where we can dump tests'}>]

In [6]:
input_dir = "company-matching__06-26-23_11-40-51"
connection = duckdb.connect()
data = du.build_alias_path_dict(input_dir)

linker = DuckDBLinker(
    list(data.values()),
    settings_dict=settings,
    connection=connection,
    input_table_aliases=list(data.keys()),
)

In [27]:
x = duckdb.sql(f"""
    select
        count(*)
    from
        read_parquet({list(data.values())})
""").fetchall()[0][0]

x

353960

In [22]:
for table in data.keys():
    duckdb.read_parquet(data[table])
    duckdb.sql(f"""
        select
            count(*)
        from
            table
    """)

ParserException: Parser Error: syntax error at or near "table"
LINE 5:             table
                    ^

In [87]:
# logger = logging.getLogger(__name__)
datasets = CompanyMatchingDatasets(sample=100)

AttributeError: 'CompanyMatchingDatasets' object has no attribute 'logger'

In [62]:
datasets.datasets_and_readfuncs_clean['companieshouse_companies'].columns
datasets.datasets_and_readfuncs_clean['dit_data_hub__companies'].columns
datasets.datasets_and_readfuncs_clean['hmrc_trade__exporters'].columns
datasets.datasets_and_readfuncs_clean['dit_export_wins__wins_dataset'].columns

Index(['unique_id', 'company_number', 'company_name', 'secondary_names',
       'postcode'],
      dtype='object')

Index(['unique_id', 'company_number', 'company_name', 'secondary_names',
       'postcode'],
      dtype='object')

Index(['unique_id', 'company_number', 'company_name', 'secondary_names',
       'postcode'],
      dtype='object')

Index(['unique_id', 'company_number', 'company_name', 'secondary_names',
       'postcode'],
      dtype='object')

In [74]:
# linker = DuckDBLinker(
#     list(datasets.datasets_and_readfuncs_clean.keys()),
#     settings_dict = settings,
#     connection = datasets.connection
# )
linker = datasets.linker(settings)
linker.estimate_probability_two_random_records_match(
    "l.name_unusual_tokens = r.name_unusual_tokens",
    recall=0.7,
)

Probability two random records match is estimated to be  3.58e-05.
This means that amongst all possible pairwise record comparisons, one in 27,930.00 are expected to match.  With 79,800 total possible comparisons, we expect a total of around 2.86 matching pairs


In [10]:
datasets.data[0].__class__

pandas.core.frame.DataFrame

In [12]:
datasets.alias

['self.companieshouse_companies',
 'self.dit_data_hub__companies',
 'self.hmrc_trade__exporters',
 'self.dit_export_wins__wins_dataset']

In [15]:
datasets.companieshouse_companies.head(1)

,unique_id,company_number,company_name,secondary_names,company_status,account_category,address_line_1,address_line_2,post_town,county,country,postcode,sic_code_1,sic_code_2,sic_code_3,sic_code_4
0,08230106,08230106,BOURNE HILL STABLES LIMITED,[TEN DELTA LIMITED],Active,MICRO ENTITY,9 CHEAM ROAD,,EPSOM,,ENGLAND,KT17 1SP,01629 - Support activities for animal producti...,,,


In [16]:
duckdb.sql("""
    select
        *
    from
        datasets.companieshouse_companies
    limit 1
""")

CatalogException: Catalog Error: Table with name companieshouse_companies does not exist!
Did you mean "temp.information_schema.columns"?

In [40]:
con = duckdb.connect()

In [52]:
# Load data
datasets = CompanyMatchingDatasets(sample=100)

# Instantiate linker
# linker = datasets.linker(settings)

table_names = []
for table in datasets.datasets_and_readfuncs_clean:
    # table_names.append(f"{table}")
    con.register(
        f"{table}", 
        datasets.datasets_and_readfuncs_clean[table]
    )


In [53]:
list(datasets.datasets_and_readfuncs_clean.keys())

['companieshouse_companies',
 'dit_data_hub__companies',
 'hmrc_trade__exporters',
 'dit_export_wins__wins_dataset']

In [54]:
# Instantiate linker
linker = DuckDBLinker(
    list(datasets.datasets_and_readfuncs_clean.keys()),
    settings_dict = settings,
    connection = con
)

In [56]:
linker.estimate_probability_two_random_records_match(
    "l.name_unusual_tokens = r.name_unusual_tokens",
    recall=0.7,
)

SplinkException: Error executing the following sql for table `__splink__df_count_cumulative_blocks`(__splink__df_count_cumulative_blocks_4c42918e5):
CREATE TABLE __splink__df_count_cumulative_blocks_4c42918e5 AS
(
  WITH __splink__df_concat AS (
    SELECT
      *
    FROM __splink__df_concat_9f1d2ff43
  ), __splink__df_blocked_data AS (
    SELECT
      "l"."source_dataset" AS "source_dataset_l",
      "r"."source_dataset" AS "source_dataset_r",
      "l"."unique_id" AS "unique_id_l",
      "r"."unique_id" AS "unique_id_r",
      "l"."comp_num_clean" AS "comp_num_clean_l",
      "r"."comp_num_clean" AS "comp_num_clean_r",
      "l"."name_unusual_tokens" AS "name_unusual_tokens_l",
      "r"."name_unusual_tokens" AS "name_unusual_tokens_r",
      "l"."postcode" AS "postcode_l",
      "r"."postcode" AS "postcode_r",
      "l"."secondary_name_unusual_tokens" AS "secondary_name_unusual_tokens_l",
      "r"."secondary_name_unusual_tokens" AS "secondary_name_unusual_tokens_r",
      '0' AS 

In [57]:
con.execute("""
    select
        *
    from
        companieshouse_companies
    limit 1
""")


In [33]:
duckdb.sql(f"""
    select
        *
    from
        {table_names[0]}
    limit 1
""")

┌───────────┬────────────────┬──────────────────────┬─────────────────┬───┬────────────┬────────────┬────────────┐
│ unique_id │ company_number │     company_name     │ secondary_names │ … │ sic_code_2 │ sic_code_3 │ sic_code_4 │
│  varchar  │    varchar     │       varchar        │    varchar[]    │   │  varchar   │  varchar   │  varchar   │
├───────────┼────────────────┼──────────────────────┼─────────────────┼───┼────────────┼────────────┼────────────┤
│ 03232349  │ 03232349       │ CLASSIC MARQUEES L…  │ []              │ … │            │            │            │
├───────────┴────────────────┴──────────────────────┴─────────────────┴───┴────────────┴────────────┴────────────┤
│ 1 rows                                                                                    16 columns (7 shown) │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘